In [1]:
import pandas as pd
import yaml

In [2]:
class node:

    def __init__(self):
        self.up = self
        self.down = self
        self.left = self
        self.right = self
        self.COL = None
        self.ROW = None

    def hideInCol(self):
        self.up.down = self.down
        self.down.up = self.up
        self.COL.row_count -= 1
        return self

    def showInCol(self):
        self.up.down = self
        self.down.up = self
        self.COL.row_count += 1
        return self

    def hideInRow(self):
        self.left.right = self.right
        self.right.left = self.left
        return self

    def showInRow(self):
        self.left.right = self
        self.right.left = self
        return self
    
    def deleteUnuseableRow(self):
        cur_cell = self.right 
        while cur_cell != self:
            print(cur_cell.COL.label)
            cur_cell.hideInCol()
            cur_cell = cur_cell.right
        print("done")
        self.hideInCol()
        return

    def hideWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.right.hideWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.hideInCol()
            self.right.hideWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish

    def showWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.left.showWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.showInCol()
            self.left.showWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish
    
    def getNeighbor(self, direction, by=0):
        neighbor = {
            'left'  : self.left,
            'right' : self.right,
            'up'    : self.up,
            'down'  : self.down
        }[direction]
        if by > 0:
            return neighbor.getNeighbor(direction, by = by-1)
        return neighbor




In [3]:
class columnHeader(node):

    def __init__(self, label):
        super().__init__()
        self.label = label
        self.row_count = 0
        self.COL = self

    def hideMe(self):
        self.hideInRow()
        cell = self.down
        while cell != self:
            cell.hideWholeRow()
            cell = cell.down
        return self

    def showMe(self):
        self.showInRow()
        cell = self.up
        while cell != self:
            cell.showWholeRow()
            cell = cell.up
        return self

    def appendRow(self, row):
        row.down = self
        row.up = self.up
        self.up.down = row
        self.up = row
        row.COL = self
        self.row_count += 1
        return self

    def addToRoot(self, root):
        root.appendColumn(self)
        return self

In [4]:
class rowHeader(node):
    
    def __init__(self, label):
        super().__init__()
        self.label = label
        self.ROW = self

    def appendColumn(self, col):
        col.right = self
        col.left = self.left
        self.left.right = col
        self.left = col
        col.ROW = self
        return self

    def addToRoot(self, root):
        root.appendRow(self)
        return self


In [5]:
class rootNode(columnHeader, rowHeader):

    def __init__(self):
        node.__init__(self)
        self.row_count = 0
        self.label = None
        self.ROW = self
        self.COL = self

    def addToRoot(self, root):
        pass

In [6]:
def append_dict(d, e):
    r = d.copy()
    r.update(e)
    return r

In [7]:
def check_simple_criterion(sparse_matrix_row_attr, check_attr):
    
    has_dims = {k for k in sparse_matrix_row_attr}
    needs_dims = {k for k in check_attr}
    overlap_attrs = has_dims & needs_dims
    can_be_checked = len(overlap_attrs) == len(check_attr)

    # not enough attributes present to perform the  check => default to OK
    if not can_be_checked:
        return True

    # check on each dimension OK => all OK
    summary_check_result = { sparse_matrix_row_attr[d] == check_attr[d] for d in overlap_attrs } 

    return summary_check_result == {True} or summary_check_result == {False}



In [8]:
def check_set_of_criteria(sparse_matrix_row_attr, check_set):

    summary_check_results = {
        check_simple_criterion(sparse_matrix_row_attr, item)
        for item in check_set
    }
    
    return summary_check_results == {True}

In [9]:
riddle_file = "input_1_einstein.yml"

In [10]:
with open(riddle_file) as f:
    try:
        riddle_text= yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)

In [11]:
tmp1 = [ 
    {"type" : "_is",
    "params" : [{"dim" : dim_name, "val" : dim_val} for dim_name, dim_val in is_condition.items()]}
     for is_condition in riddle_text if len(is_condition) > 1
]
equalities_list = [
    {
        "type" : i["type"],
        "first" : i["params"][0],
        "second" : i["params"][1]
    }    
    for i in tmp1
]
equalities_list

[{'type': '_is',
  'first': {'dim': 'nation', 'val': 'Norway'},
  'second': {'dim': '_#', 'val': 1}},
 {'type': '_is',
  'first': {'dim': 'nation', 'val': 'England'},
  'second': {'dim': 'house_color', 'val': 'red'}},
 {'type': '_is',
  'first': {'dim': 'nation', 'val': 'Dutch'},
  'second': {'dim': 'drink', 'val': 'tea'}},
 {'type': '_is',
  'first': {'dim': 'cig', 'val': 'cigars'},
  'second': {'dim': 'house_color', 'val': 'yellow'}},
 {'type': '_is',
  'first': {'dim': 'nation', 'val': 'Germany'},
  'second': {'dim': 'cig', 'val': 'pipe'}},
 {'type': '_is',
  'first': {'dim': '_#', 'val': 3},
  'second': {'dim': 'drink', 'val': 'milk'}},
 {'type': '_is',
  'first': {'dim': 'cig', 'val': 'no_filter'},
  'second': {'dim': 'pet', 'val': 'birds'}},
 {'type': '_is',
  'first': {'dim': 'nation', 'val': 'Sweden'},
  'second': {'dim': 'pet', 'val': 'dogs'}},
 {'type': '_is',
  'first': {'dim': 'cig', 'val': 'menthol'},
  'second': {'dim': 'drink', 'val': 'beer'}},
 {'type': '_is',
  'first'

In [12]:
tmp2 = [
    {
        "type" : condition_name,
        "params" : [{"dim": dim_name, "val" : dim_val} for i in condition_params for dim_name, dim_val in i.items()],
    }
    for neighbor_condition in riddle_text if len(neighbor_condition) == 1
        for condition_name, condition_params in neighbor_condition.items() if condition_name != '_question'
]
neighbors_list = [
    {
        "type" : i["type"],
        "first" : i["params"][0],
        "second" : i["params"][1]
    }    
    for i in tmp2
]
neighbors_list

[{'type': '_left',
  'first': {'dim': 'house_color', 'val': 'green'},
  'second': {'dim': 'house_color', 'val': 'white'}},
 {'type': '_adj',
  'first': {'dim': 'cig', 'val': 'light'},
  'second': {'dim': 'pet', 'val': 'cat'}},
 {'type': '_adj',
  'first': {'dim': 'cig', 'val': 'light'},
  'second': {'dim': 'drink', 'val': 'water'}},
 {'type': '_adj',
  'first': {'dim': 'nation', 'val': 'Norway'},
  'second': {'dim': 'house_color', 'val': 'blue'}},
 {'type': '_adj',
  'first': {'dim': 'house_color', 'val': 'yellow'},
  'second': {'dim': 'pet', 'val': 'horses'}}]

In [13]:
other_values = [
    {
        "condition" : condition_name,
        "dim": dim_name,
        "val": value
    }
    for other_condition in riddle_text if len(other_condition) == 1
        for condition_name, condition_param_list in other_condition.items() if condition_name == '_question'
    for condtion_param in condition_param_list
    for dim_name, value in condtion_param.items()
]
other_values

[{'condition': '_question', 'dim': 'pet', 'val': 'fish'}]

In [14]:
all_items = [e["first"] for e in equalities_list] + \
[e["second"] for e in equalities_list] + \
[e["first"] for e in neighbors_list] + \
[e["second"] for e in neighbors_list] + \
other_values

all_items



[{'dim': 'nation', 'val': 'Norway'},
 {'dim': 'nation', 'val': 'England'},
 {'dim': 'nation', 'val': 'Dutch'},
 {'dim': 'cig', 'val': 'cigars'},
 {'dim': 'nation', 'val': 'Germany'},
 {'dim': '_#', 'val': 3},
 {'dim': 'cig', 'val': 'no_filter'},
 {'dim': 'nation', 'val': 'Sweden'},
 {'dim': 'cig', 'val': 'menthol'},
 {'dim': 'house_color', 'val': 'green'},
 {'dim': '_#', 'val': 1},
 {'dim': 'house_color', 'val': 'red'},
 {'dim': 'drink', 'val': 'tea'},
 {'dim': 'house_color', 'val': 'yellow'},
 {'dim': 'cig', 'val': 'pipe'},
 {'dim': 'drink', 'val': 'milk'},
 {'dim': 'pet', 'val': 'birds'},
 {'dim': 'pet', 'val': 'dogs'},
 {'dim': 'drink', 'val': 'beer'},
 {'dim': 'drink', 'val': 'coffee'},
 {'dim': 'house_color', 'val': 'green'},
 {'dim': 'cig', 'val': 'light'},
 {'dim': 'cig', 'val': 'light'},
 {'dim': 'nation', 'val': 'Norway'},
 {'dim': 'house_color', 'val': 'yellow'},
 {'dim': 'house_color', 'val': 'white'},
 {'dim': 'pet', 'val': 'cat'},
 {'dim': 'drink', 'val': 'water'},
 {'dim'

In [15]:
dicts = {
    i["dim"] : set()
    for i in all_items
}

[
    dicts[i["dim"]].add(i["val"])
    for i in all_items
]
    
max_items = max(
    [
        len(v)
        for  v in dicts.values()
    ]
)
dicts['_#'] = {
    r+1
    for r in range(max_items)
}

dicts

{'nation': {'Dutch', 'England', 'Germany', 'Norway', 'Sweden'},
 'cig': {'cigars', 'light', 'menthol', 'no_filter', 'pipe'},
 '_#': {1, 2, 3, 4, 5},
 'house_color': {'blue', 'green', 'red', 'white', 'yellow'},
 'drink': {'beer', 'coffee', 'milk', 'tea', 'water'},
 'pet': {'birds', 'cat', 'dogs', 'fish', 'horses'}}

In [16]:
simple_guards = [item for item in riddle_text if len(item) !=1] 

In [17]:
simple_guards

[{'nation': 'Norway', '_#': 1},
 {'nation': 'England', 'house_color': 'red'},
 {'nation': 'Dutch', 'drink': 'tea'},
 {'cig': 'cigars', 'house_color': 'yellow'},
 {'nation': 'Germany', 'cig': 'pipe'},
 {'_#': 3, 'drink': 'milk'},
 {'cig': 'no_filter', 'pet': 'birds'},
 {'nation': 'Sweden', 'pet': 'dogs'},
 {'cig': 'menthol', 'drink': 'beer'},
 {'house_color': 'green', 'drink': 'coffee'}]

{1: 3, '5': 3}

In [263]:
class einstein_matrix:

    def __init__(self):

        self.matrix_root = rootNode()
        self.simple_column_headers = {}
        self.row_headers = []
        self.neighbor_col_headers = []

    def create_columns(self, dicts):

        self.simple_column_headers = {
            dim :{val : columnHeader(f"{dim}:{val}").addToRoot(self.matrix_root) for val in vals} for dim, vals in dicts.items()
        }

        return self

    def create_neighbor_columns(self, neighbor_params, number_of_houses):

        for cond in neighbor_params:
            direction = cond["type"]
            for i in range(number_of_houses):
                house_index = i+1
                label_txt = f"{house_index} : {cond["first"]["dim"]}:{cond["first"]["val"]} {direction} {cond["second"]["dim"]}:{cond["second"]["val"]}"
                label_dict = {
                    "house_index" : house_index,
                    "first_dim" : cond["first"]["dim"],
                    "first_val" : cond["first"]["val"],
                    "second_dim" : cond["second"]["dim"], 
                    "second_val" : cond["second"]["val"]
                }
                header_node = columnHeader(label_txt).addToRoot(self.matrix_root)
                self.neighbor_col_headers.append(
                    {
                        "attrs" : label_dict,
                        "rcol_header" :  header_node
                    }
                )
        return self

    def create_row_labels(self, dicts, guards):
    
        level = 0
        labels = [{}]
        copy_dics = dicts.copy()

        while len(copy_dics) > 0:
            dim = copy_dics.popitem()
            dim_name = dim[0]
            dim_values = dim[1]
            labels = [
                append_dict(label, {dim_name: value})
                for label in labels 
                for value in dim_values
                if check_set_of_criteria(
                    append_dict(label, {dim_name: value}),
                guards)
            ]

        return labels

    def create_rows(self, dicts, guards):
        
        labels = self.create_row_labels(dicts, guards)

        self.row_headers = [
            {
                "attrs" : l,
                "row_header" :
                    rowHeader(
                        ":".join(
                            [
                                str(i) for i in list(l.values())
                            ]
                        )
                    ).addToRoot(self.matrix_root)
                
            }   for l in labels
        ]

        return self





In [124]:
(1, 2) == (2,1)

False

In [284]:
em = einstein_matrix()
em.create_columns(dicts) \
   .create_rows(dicts, simple_guards) \
   .create_neighbor_columns(neighbors_list, max_items)
[i for i in em.neighbor_col_headers
if
   True
   # and i["attrs"]["house_index"] == 1
   and i["attrs"]["second_dim"] == "pet"
   and i["attrs"]["second_val"] == "horses"
]


[{'attrs': {'house_index': 1,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7ef1e81666c0>},
 {'attrs': {'house_index': 2,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7ef1e8166240>},
 {'attrs': {'house_index': 3,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7ef1e8166750>},
 {'attrs': {'house_index': 4,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7ef1e8166420>},
 {'attrs': {'house_index': 5,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7ef1e8164e60>}]

In [21]:
def search(matrix, solution=[], log=[], k=0):

    # print(f"k : {k}")

    if type(matrix) != rootNode:
        raise TypeError("Must start with matrix root node")

    # solution found if matrix is empty
    if matrix.right == matrix:
        print("ready")
        return prep_solution(solution)

    # select column
    col = matrix.right
    tmp_col = col
    while tmp_col != matrix:
        if tmp_col.row_count < col.row_count:
            col = tmp_col
        tmp_col = tmp_col.right
    print(col.label)
    # hide column 
    col.hideMe()
    # go through its each row
    row = col.down
    print(row.ROW.label)
    while row != col:
        # add row to solution
        solution.append(row)
        log.append(f"select R{row.ROW.label['R']}C{row.ROW.label['C']}={row.ROW.label['#']}")
        # hide row's columns
        cell = row.right
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.hideMe()
            cell = cell.right
        # go deeper
        result = search(matrix, solution, log, k = k+1)
        if result != None:
            return result
        # uncover columns and remove row from solution
        solution.pop()
        log.append("back")
        cell = row.left
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.showMe()
            cell = cell.left
        # next row
        row = row.down
    # # uncover column
    col.showMe()
    return None

    